<a href="https://colab.research.google.com/github/krishna1072/JanataHack-Recommendation-Systems/blob/master/NN_Recommendation_Systems.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import zipfile
import gensim
from random import sample

In [ ]:
# Load train data
zf = zipfile.ZipFile('/content/train_mddNHeX.zip') # having First.csv zipped file.
train_chal = pd.read_csv(zf.open('train.csv'))
display(train_chal.head())
train_chal.info()

# Load all the challenges
challenges = pd.read_csv(zf.open('challenge_data.csv'))
challenges['publish_date'] = pd.to_datetime(challenges['publish_date'])
display(challenges.head())
challenges.info()

# Load test data
zf = zipfile.ZipFile('/content/test_HLxMpl7.zip') # having First.csv zipped file.
test_chal = pd.read_csv(zf.open('test.csv'))
display(test_chal.head())
test_chal.info()

In [ ]:
train=train_chal.groupby('user_id').challenge.apply(lambda x: ' '.join(x)).reset_index()

In [ ]:
test=test_chal.groupby('user_id').challenge.apply(lambda x: ' '.join(x)).reset_index()

In [ ]:
set(train.user_id).intersection(test.user_id)

In [ ]:
train.shape, test.shape

In [ ]:
sentence_list=pd.concat((train.challenge,test.challenge)).apply(lambda x: x.split())
# sentence_list= train.challenge.apply(lambda x: x.split())

In [ ]:
# train word2vec model to learn embeddings
neg=15
itera=100
window=5
size=500
print ('doing_'+str(neg)+'_'+str(window)+'_'+str(itera))
model=gensim.models.Word2Vec(sentence_list,size=size,window=window,workers=8,min_count=0, hs=1,iter=itera,sg=1)
model.save('word2vec_solved_iter__noshuf_5window_'+ str(itera) + '_size_' + str(size))

In [ ]:
model= gensim.models.Word2Vec.load('word2vec_solved_iter__noshuf_5window_100_size_500')
model.most_similar('CI23855')

In [ ]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers import TimeDistributed, concatenate, Bidirectional, Masking, RepeatVector #,Merge
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, GRU, SimpleRNN
from keras.layers.convolutional import Convolution1D, MaxPooling1D

In [ ]:
pretrained_weights = model.wv.syn0
vocab_size, emdedding_size = pretrained_weights.shape
print('Result embedding shape:', pretrained_weights.shape)

In [ ]:
def word2idx(word):
    return model.wv.vocab[word].index
def idx2word(idx):
    return model.wv.index2word[idx]

In [ ]:
# generate int sequences for challenge sentences
train_seq=np.array(train.challenge.apply(lambda x: x.split()).apply(lambda x: ([(word2idx(y)) for y in x])).tolist())
test_seq=np.array(test.challenge.apply(lambda x: x.split()).apply(lambda x: ([(word2idx(y)) for y in x])).tolist())

In [ ]:
xtrain=train_seq[:,:10]
ytrain= train_seq[:,10]
xtrain.shape, ytrain.shape

In [ ]:
# add 11th,12th,13th challenge to outcome
xtrain= np.concatenate((xtrain,xtrain,xtrain),axis=0)
ytrain= np.concatenate((train_seq[:,10],train_seq[:,11],train_seq[:,12]))

In [ ]:
# define nn model
emdedding_size=500
vocab_size=5502
keras_model2 = Sequential()
keras_model2.add(Embedding(input_dim=vocab_size, output_dim=emdedding_size, 
                    weights=[pretrained_weights]))
keras_model2.add(GRU(units=emdedding_size))
keras_model2.add(Dense(units=vocab_size))

keras_model2.add(Activation('softmax'))
keras_model2.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

In [ ]:
sample_weight=np.concatenate((np.ones(train.shape[0])*3,np.ones(train.shape[0])*2,np.ones(train.shape[0])))

In [ ]:
# fit differnt samples of data and then full data
keras_model2.fit(xtrain,ytrain, epochs=2, sample_weight=sample_weight,
          batch_size=512, verbose=1,validation_split=0.11)
keras_model2.fit(xtrain,ytrain, epochs=1, sample_weight=sample_weight,
          batch_size=512, verbose=1,validation_split=0.11)
keras_model2.fit(xtrain,ytrain, epochs=1, sample_weight=sample_weight,
          batch_size=512, verbose=1,validation_split=0.11)
keras_model2.fit(xtrain,ytrain, epochs=1, sample_weight=sample_weight,
          batch_size=512, verbose=1)

In [ ]:
import pickle
pickle.dump(keras_model2.to_json(),open('gru4.pkl','wb'))

In [ ]:
import h5py
import keras
# from importlib import reload
# import keras
keras_model2.save_weights('gru4.weights')

In [ ]:
def sample(preds, temperature=1.0):
    if temperature <= 0:
        return preds.argsort()[-3:][::-1]
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
#     print max(probas[0])
    retar= probas[0].argsort()[-3:][::-1]
    return retar

def generate_next(text):
    # global count
    # count+=1
    # if count%100==0:
    #     print (count)
    word_idxs = [word2idx(word) for word in text.split()]
    prediction = keras_model2.predict(x=np.array(word_idxs))
#     print prediction
    idxl = sample(prediction[-1], temperature=0.0)
#     word_idxs.e(idx)
#     print idxl
    return [idx2word(idx) for idx in idxl]



In [ ]:
test.challenge.values[0]

In [ ]:
generate_next('CI23855 CI23933 CI24917 CI24915 CI23714 CI23663 CI24958 CI25135 CI25727 CI24530')

In [ ]:
test_inds=np.array(test.challenge.apply(lambda x: np.array([word2idx(word) for word in x.split()])).tolist())

In [ ]:
test_inds.shape, xtrain.shape

In [ ]:
# make predictions
nnpred=keras_model2.predict(test_inds)

In [ ]:
nnpred.shape

In [ ]:
nnpred2= pd.Series(nnpred.argsort(axis=1)[:,-3:].tolist())

In [ ]:
# store to csv
count=0
test['pred']=nnpred2.apply(lambda x: [idx2word(y) for y in x[::-1]])
test_sub= test.copy()
test_sub=pd.concat((test_sub,test_sub,test_sub))
test_sub=test_sub.sort_values('user_id').reset_index(drop=True)
test_sub['seq']=test_sub.groupby('user_id').cumcount()
test_sub.seq= test_sub.seq+11
test_sub['user_sequence']= test_sub.user_id.astype('str') + '_' + test_sub.seq.astype('str')
test_sub['challenge']=test_sub.apply(lambda row: row['pred'][row['seq']-11], axis=1)
test_sub[['user_sequence','challenge']].to_csv('/content/NN_submission_06282020.csv',index=False)